In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
from transformers import pipeline

In [2]:
# Load GPT-2 model and tokenizer
model_name = "gpt2"  # You can also use "gpt2-medium", "gpt2-large", etc.
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
# Prepare the dataset
def load_dataset(train_file, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path="/content/sample_data/train.txt",
        block_size=block_size,
    )
    return dataset

def load_data_collator(tokenizer):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

# Define the file path to your dataset
train_file = "/content/sample_data/train.txt"  # Replace with your text file

# Load the dataset and data collator
train_dataset = load_dataset(train_file, tokenizer)
data_collator = load_data_collator(tokenizer)

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",  # Output directory
    overwrite_output_dir=True,
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=2,  # Batch size
    save_steps=10_000,  # Save checkpoint every 10,000 steps
    save_total_limit=2,  # Limit the total amount of checkpoints
    logging_steps=200,  # Log every 200 steps
    report_to="none",  # Disable W&B logging
)

In [7]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()

# Save the model and tokenizer
trainer.save_model("./gpt2-finetuned") # Saves the model weights
tokenizer.save_pretrained("./gpt2-finetuned") # Saves the tokenizer

# Testing the fine-tuned model
generator = pipeline('text-generation', model='./gpt2-finetuned')
output = generator("The fine-tuned model says:", max_length=50)
print(output)

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'The fine-tuned model says: "In the case of a small group with small incomes, there are no barriers to employment."\n\nThe research was published in the journal Economic Inquiry in February.\n\nThe study, which was funded by the European Commission and the National Institute for Employment and Growth, also found that immigrants, particularly those with the highest incomes, are more likely to be employed outside the EU.\n\n"They are more likely to be employed in the EU," said Professor Andrew Macdonald from the University of Glasgow\'s School of Business.\n\n"There are also a number of advantages to the small groups that are here. Many of them are low- and middle-earning.\n\n"Many of them are in the UK, and the UK is the largest economy in Europe.\n\n"One of the main things we found is that many of the small groups were very unlikely to be able to be employed in the UK.\n\n"We could not exclude the possibility that the small groups may have been exposed to a range of